In [1]:
import numpy as np

from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.visualise_layouts_and_metrics.plot import show_layout
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from experiments.utils.get_kmeans_f1 import get_kmeans_f1
from experiments.utils.get_f1_for_best_k_with_knn import get_f1_for_best_k_with_knn
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle
from scipy import stats
import matplotlib.pyplot as plt
from time import perf_counter
from  sklearn.manifold import SpectralEmbedding, MDS
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean
import pandas as pd

In [6]:
dataset= DataFetcher.fetch_data("metro", size=50000)

####################
Fetching the "metro" dataset
[[ 8.364  8.35  -0.02  ...  0.     1.     1.   ]
 [ 9.382  9.37  -0.02  ...  0.     1.     1.   ]
 [ 9.998  9.99  -0.024 ...  0.     1.     1.   ]
 ...
 [ 5.104 -0.01  -0.02  ...  1.     1.     1.   ]
 [ 8.966 -0.014 -0.016 ...  0.     1.     1.   ]
 [ 8.43   8.422 -0.018 ...  0.     1.     1.   ]]
####################
Dataset loaded
Dataset shape: (50000, 13)
####################


In [7]:
results = [[],[]]
num_repeats = 2

In [8]:
Xld = PCA(n_components=2, whiten=False, copy=True).fit_transform(dataset.data).astype(np.float64)
Xld *= 10/np.std(Xld)

for i in range(num_repeats):

    no_knnd_start = perf_counter()
    algo96 = Chalmers96(dataset=dataset, distance_fn=euclidean,
             spring_constant=0.05, initial_layout=Xld,
            use_knnd=False)
    layout = LayoutCreation.create_layout(algo96, num_iters=100)
    results[0].append(perf_counter() - no_knnd_start)



    knnd_start = perf_counter()
    algo96 = Chalmers96(dataset=dataset, distance_fn=euclidean,
             spring_constant=0.05, initial_layout=Xld,
            use_knnd=True)
    layout_knnd = LayoutCreation.create_layout(algo96, num_iters=100)
    results[1].append(perf_counter() - knnd_start)



####################
A 2D layout of the "metro" dataset will be created 
using the "Chalmers' 1996" algorithm
####################
The HD distance measure used is: euclidean
####################
####################
No metrics will be collected during layout creation. 
To change this use the 'metric collection' parameter of the layout 
####################
All stress calculations will be performed using the euclidian norm
####################
Spring constant is set to  0.05 
Damping constant is set to 0 
Spring constant scaling factor is set to 0.03333333333333333 
####################


KeyboardInterrupt: 

In [9]:
results = np.array([[296.41819789999863, 308.94675129999814], [480.133877100001, 392.8605233999988]])

In [10]:
means = np.mean(results, axis=1)
print(means[1]- means[0])
print((means[1]- means[0])/means[1])

133.8147256500015
0.30656491169556255
